# TO SPLIT THE FOLDER AND DO THE TRAIN TEST AND VALIDATION SPLIT

In [1]:
!pip install split_folders

In [1]:
import splitfolders

In [2]:
input_folder = 'Flowers_dataset/Input_dataset'
output_folder = 'Flowers_dataset/processed_data'
splitfolders.ratio(input_folder,output_folder,seed=42,ratio=(.6,.2,.2))

# for training , testing and validation in processed_data folder

Copying files: 4326 files [02:28, 29.08 files/s]


In [3]:
help(splitfolders.ratio)

Help on function ratio in module splitfolders.split:

ratio(input, output='output', seed=1337, ratio=(0.8, 0.1, 0.1), group_prefix=None)



# TRAIN THE MODEL ON FLOWER RECOGNITION DATASET 

In [4]:
from tensorflow.keras.applications.resnet50 import ResNet50,decode_predictions, preprocess_input #taking pretrained weights fromkeras
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from keras.datasets import cifar10
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import time
#from skimage.transform import resize
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator,load_img

In [5]:
height,width = (224,224)
batch_size=32

train_dir = r'Flowers_dataset/processed_data/train'
test_dir = r'Flowers_dataset/processed_data/test'
val_dir = r'Flowers_dataset/processed_data/val'

In [6]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
training_set

Found 2593 images belonging to 5 classes.


In [7]:
testing_set = train_datagen.flow_from_directory(test_dir,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')
testing_set

Found 870 images belonging to 5 classes.


In [8]:
valid_set = train_datagen.flow_from_directory(val_dir,
                                            target_size = (224, 224),
                                            batch_size = 1,
                                            class_mode = 'categorical')
valid_set

Found 860 images belonging to 5 classes.


In [9]:
x,y = valid_set.next()
x.shape

(1, 224, 224, 3)

In [10]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [11]:
## functional API

x = resnet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(50, activation='relu')(x)
predictions= Dense(training_set.num_classes,activation = 'softmax')(x)

model = Model(inputs = resnet_model.input,outputs =predictions )

# don't train existing weights
for layer in resnet_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [12]:
r = model.fit(testing_set,epochs = 5,verbose=1)

Epoch 1/5
28/28 [==============================] - 26s 911ms/step - loss: 0.8210 - accuracy: 0.7069
Epoch 2/5
28/28 [==============================] - 17s 592ms/step - loss: 0.3074 - accuracy: 0.8851
Epoch 3/5
28/28 [==============================] - 19s 693ms/step - loss: 0.1706 - accuracy: 0.9506
Epoch 4/5
28/28 [==============================] - 19s 689ms/step - loss: 0.1101 - accuracy: 0.9759
Epoch 5/5
28/28 [==============================] - 19s 672ms/step - loss: 0.0726 - accuracy: 0.9897


In [13]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

conv5_block3_3_conv (Conv2D)    (None, 7, 7, 2048)   1050624     conv5_block3_2_relu[0][0]        
__________________________________________________________________________________________________
conv5_block3_3_bn (BatchNormali (None, 7, 7, 2048)   8192        conv5_block3_3_conv[0][0]        
__________________________________________________________________________________________________
conv5_block3_add (Add)          (None, 7, 7, 2048)   0           conv5_block2_out[0][0]           
                                                                 conv5_block3_3_bn[0][0]          
__________________________________________________________________________________________________
conv5_block3_out (Activation)   (None, 7, 7, 2048)   0           conv5_block3_add[0][0]           
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 2048)         0           conv5_block3_out[0][0]           
__________

In [14]:
model.save('flower_model.h5')

In [ ]:
test_loss, test_acc = model.evaluate(valid_set,verbose = 2)
print('Test Accuracy : ',test_acc)

In [18]:
_,train_acc = model.evaluate(training_set,verbose = 2)
print('Training Accuracy',train_acc)

82/82 - 72s - loss: 0.3845 - accuracy: 0.8916
Training Accuracy 0.8916313052177429


In [15]:
from keras.preprocessing import image
from keras.models import load_model

test_image = image.load_img('Flowers_dataset/processed_data/val/rose/rose.jpg',
                            target_size = (224,224))

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

model = load_model('flower_model.h5')
result = model.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'NOT A ROSE'
    print(prediction)
else:
    prediction = 'ROSE'
    print(prediction)

ROSE


In [16]:
from keras.preprocessing import image
from keras.models import load_model

test_image = image.load_img('Flowers_dataset/processed_data/val/sunflower/sunflower.jpg',
                            target_size = (224,224))

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

model = load_model('flower_model.h5')
result = model.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'NOT A SUNFLOWER'
    print(prediction)
else:
    prediction = 'SUNFLOWER'
    print(prediction)

SUNFLOWER
